<a href="https://colab.research.google.com/github/pi-aire/dataminig_sampling/blob/main/TP_DataMining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Échantillonnage direct de l'espace des motifs

### BRUNEAU Richard - VASLIN Pierre





In [ ]:
import numpy as np
import scipy.special as sps
import math
import matplotlib
import pandas as pd
import random

In [ ]:
# Le dataframe
df = pd.read_table("https://bitbucket.org/anesbendimerad/sigibbssamplingcode/raw/6699a50508fe177ee0c00dcc7d8e5390ee53688a/ItemsetDatasets/chess.txt", sep=" ",header=None)
del df[37]
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36
0,1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74
1,1,3,5,7,9,12,13,15,17,19,21,23,25,27,29,31,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74
2,1,3,5,7,9,12,13,16,17,19,21,23,25,27,29,31,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74
3,1,3,5,7,9,11,13,15,17,20,21,23,25,27,29,31,34,36,38,40,42,44,47,48,50,52,54,56,58,60,62,64,66,68,70,72,74
4,1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31,34,36,38,40,42,44,46,48,51,52,54,56,58,60,62,64,66,68,70,72,74


## Question 1

In [ ]:
def algoFrequences(df:pd.DataFrame,nb_pattern)-> tuple:
  #R:set = set()
  R = []
  IsInR = set()
  P: dict = dict()
  w = np.zeros(df[0].count())
  totalW = 0
    
  # set les probas
  for i in range(len(w)):
    w[i] = math.pow(2,len(df.iloc[i]))
    totalW += w[i]
  
  # on selectionne 
  while len(R) < nb_pattern:
    random_row = random.uniform(0,totalW)
    # On cherche la ligne
    row = 0
    v = 0
    for i in range(len(w)):
      if  v > random_row:
        row = i - 1
        break
      v += w[i]
    # On selctionne un motif 
    pattern = np.array(df.iloc[row])
    random_v = random.randint(1, len(df.iloc[row]) - 1 )
    for i in range(len(pattern)- random_v):
      pattern = np.delete(pattern, random.randint(0, len(pattern) - 1 ))
    # On ajoute seulement les motifes non présent dans l'ensemble R
    IsInR.add(np.array2string(pattern))
    if len(IsInR) != len(R):
      R.append(pattern)
  return R

In [ ]:
algoFrequences(df,10)

## Question 2

In [ ]:
def algoArea(df:pd.DataFrame,nb_pattern)-> tuple:
  R = []
  IsInR = set()
  
  # set les probas
  w = np.zeros(df[0].count())
  totalW = 0
  for i in range(1, len(w)):
    w[i] = len(df.iloc[i]) * (2 ** (len(df.iloc[i]) - 1))
    totalW += w[i]
  
  # on selectionne 
  while len(R) < nb_pattern:
    # On cherche la ligne
    random_row = random.uniform(0,totalW)
    v, row = 0,0
    for i in range(len(w)):
      if v > random_row:
        row = i - 1
        break
      v += w[i]
      
    # On set les probabilités de k (taille du motif)
    # On souhaite que le sous-ensemble est une taille calculée 
    # proportionnellement avec les tailles des datarecords (ligne) 
    ks = np.zeros(len(df.iloc[row]))
    totalK = 0
    for i in range(len(df.iloc[row])):
        ks[i] = sps.binom(len(df.iloc[row]), i + 1)
        totalK += ks[i]
    random_kp = random.uniform(0, totalK)
    # On cherche la ligne
    k, v = 0,0
    for i in range(len(df.iloc[row])):
      if v > random_kp:
        k = i - 1
        break
      v += ks[i]
    
    pattern = np.array(df.iloc[row])
    for i in range(len(pattern)- k):
      pattern = np.delete(pattern, random.randint(0, len(pattern) - 1 ))
    
    # On ajoute seulement les motifs non présent dans l'ensemble R
    IsInR.add(np.array2string(pattern))
    if len(IsInR) != len(R):
      R.append(pattern)
  return R

In [ ]:
algoArea(df,10)

[array([44, 46,  2, 64, 72, 56, 60, 54, 74, 25, 72, 33, 70, 34,  3, 62, 48,
         9, 25]),
 array([31, 69, 12, 20, 12, 56, 36, 50, 55, 29, 40, 42,  9, 25, 40, 62, 58,
        56]),
 array([55, 66, 51, 70, 66, 56, 44, 64, 38,  2, 23, 36, 58, 42, 56, 66, 66,
        23, 48, 72]),
 array([ 1, 36, 50, 60, 14, 62, 23, 22, 45, 29,  9, 17, 11, 38, 28, 58, 22,
         1, 25, 52, 15, 52]),
 array([ 3, 23, 31, 36, 66, 75, 52, 40, 21, 42, 14,  3, 40, 34, 48, 60, 64,
        48, 58,  1]),
 array([23, 42, 25, 23, 74, 62,  3, 72, 52,  2, 28,  9, 62,  5, 31, 36, 16,
        16]),
 array([ 9, 19, 19, 44, 42, 54, 44, 46, 46,  7, 24, 74,  5, 38,  7, 56, 16,
        50,  7,  2]),
 array([ 4, 52, 12,  4, 66, 24, 29, 21, 48, 25, 25,  9, 21, 36, 58, 38, 66,
        62, 27, 20, 31, 38, 15, 62]),
 array([68, 68, 52, 17, 44,  5, 12, 36, 29, 17,  7, 68, 12, 17,  3, 74, 31,
        34, 60, 28,  5, 31, 70, 68, 60]),
 array([71, 48, 10, 74, 24, 36, 19, 58, 71, 58, 74, 54, 46, 69, 32,  3, 14,
        14])]

##Question 3

In [ ]:
def frequences(df, patterns):
  frequencesP = np.zeros(len(patterns),dtype=float)
  for i in range(df.count()[0]):
    indexs = np.zeros(len(patterns),dtype=int)
    for j in range(df.iloc[i].count()):
      for ip in range(len(patterns)):
        #print(df.iloc[i][j],patterns[ip][indexs[ip]])
        if (len(patterns[ip])!= indexs[ip] and 
            df.iloc[i][j] == patterns[ip][indexs[ip]]):
          indexs[ip] += 1
          if len(patterns[ip]) == indexs[ip]:
            frequencesP[ip] += 1.
        #print(indexs)
  for i in range(len(frequencesP)):
    frequencesP[i] = (frequencesP[i])/float(df.shape[0])
  return frequencesP

In [ ]:
patterns = algoFrequences(df,10)

In [ ]:
frequences(df,patterns)

array([0.00062578, 0.00187735, 0.00062578, 0.00031289, 0.00062578,
       0.00062578, 0.00156446, 0.00156446, 0.26001252, 0.25219024])

In [ ]:
def aire(df, patterns):
  aireP = np.zeros(len(patterns),dtype=float)
  for i in range(df.count()[0]):
    indexs = np.zeros(len(patterns),dtype=int)
    for j in range(df.iloc[i].count()):
      for ip in range(len(patterns)):
        #print(df.iloc[i][j],patterns[ip][indexs[ip]])
        if (len(patterns[ip])!= indexs[ip] and 
            df.iloc[i][j] == patterns[ip][indexs[ip]]):
          indexs[ip] += 1
          if len(patterns[ip]) == indexs[ip]:
            aireP[ip] += 1.
        #print(indexs)
  for i in range(len(aireP)):
    aireP[i] = aireP[i]*len(patterns[i])
  return aireP

In [ ]:
patterns = algoArea(df,10)

In [ ]:
aire(df,patterns)

array([455.,  88.,  51.,  42.,  72.,  68., 240.,  96., 777., 150.])

## Question 4

Nous allons maintenant tester nos algorithmes avec les jeux de données suggérés dans le sujet.
Dans un premier temps, nous chargeons les datasets. 

In [ ]:
dataset_1 = pd.read_table("https://bitbucket.org/anesbendimerad/sigibbssamplingcode/raw/6699a50508fe177ee0c00dcc7d8e5390ee53688a/ItemsetDatasets/chess.txt", sep=" ",header=None)
del dataset_1[37]
dataset_1.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36
0,1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74
1,1,3,5,7,9,12,13,15,17,19,21,23,25,27,29,31,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74
2,1,3,5,7,9,12,13,16,17,19,21,23,25,27,29,31,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74
3,1,3,5,7,9,11,13,15,17,20,21,23,25,27,29,31,34,36,38,40,42,44,47,48,50,52,54,56,58,60,62,64,66,68,70,72,74
4,1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31,34,36,38,40,42,44,46,48,51,52,54,56,58,60,62,64,66,68,70,72,74


In [ ]:
dataset_2 = pd.read_fwf("https://www.philippe-fournier-viger.com/spmf/datasets/kosarak_sequences.txt", sep=" ",header=None,)
dataset_2 = dataset_2[0].str.split(' ', expand=True)
dataset_2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102
0,1,-1,2,-1,3,-1,-2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,1,-1,-2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,4,-1,5,-1,6,-1,7,-1,-2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,1,-1,8,-1,-2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,9,-1,10,-1,-2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [ ]:
dataset_3 = pd.read_fwf("https://www.philippe-fournier-viger.com/spmf/datasets/MSNBC.txt", sep=" ",header=None,)
dataset_3 = dataset_3[0].str.split(' ', expand=True)
dataset_3.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56
0,6,-1,9,-1,4,-1,10,-1,3,-1,10,-1,5,-1,10,-1,4,-1,-2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,13,-1,7,-1,13,-1,7,-1,13,-1,7,-1,13,-1,7,-1,13,-1,7,-1,13,-1,14,-1,13,-1,7,-1,4,-1,7,-1,13,-1,-2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,1,-1,4,-1,7,-1,1,-1,10,-1,1,-1,2,-1,1,-1,6,-1,1,-1,-2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,9,-1,7,-1,9,-1,7,-1,9,-1,12,-1,6,-1,10,-1,13,-1,14,-1,12,-1,3,-1,-2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,4,-1,10,-1,2,-1,10,-1,12,-1,10,-1,7,-1,1,-1,10,-1,-2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [ ]:
dataset_4 = pd.read_fwf("https://www.philippe-fournier-viger.com/spmf/datasets/BIBLE.txt", sep=" ",header=None,)
dataset_4 = dataset_4[0].str.split(' ', expand=True)
dataset_4.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174
0,1,-1,2,-1,3,-1,4,-1,5,-1,6,-1,7,-1,8,-1,9,-1,10,-1,11,-1,12,-1,13,-1,14,-1,15,-1,16,-1,17,-1,18,-1,19,-1,20,-1,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,40,-1,41,-1,42,-1,43,-1,44,-1,45,-1,46,-1,47,-1,31,-1,43,-1,48,-1,45,-1,46,-1,49,-1,-2,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,31,-1,50,-1,51,-1,15,-1,52,-1,53,-1,54,-1,55,-1,56,-1,8,-1,57,-1,31,-1,56,-1,50,-1,14,-1,58,-1,59,-1,60,-1,61,-1,53,-1,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,69,-1,62,-1,70,-1,15,-1,71,-1,16,-1,72,-1,73,-1,72,-1,74,-1,75,-1,20,-1,4,-1,52,-1,76,-1,-2,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,77,-1,78,-1,51,-1,79,-1,18,-1,80,-1,22,-1,81,-1,52,-1,82,-1,46,-1,83,-1,20,-1,84,-1,33,-1,10,-1,85,-1,86,-1,73,-1,61,-1,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


## Question 5

### Etude emprique

## Question 6

On a constaté que lorsque l'on avait une ligne beaucoup plus grande que les autres, les algorithmes de fréquence et d'aire favorisent la ligne la plus grande pour créer des motifs. On souhaite trouver des motifs fréquents, donc on souhaite éviter des motifs qui s'applique seulement dans des grandes lignes. Nous avons trouver deux solutions:

*   Supprimer les grandes lignes du dataset
*   Réduire la taille des grandes lignes


`Pas sur encore:`
Nous avons choisie d'implémenter la seconde solution. On souhaite faire le traitement du dataset en un seul parcours pour éviter que le calcul soit trop long. On va donc utiliser une moyenne glissante pour évaluer si l'on doit réduire la taille de la ligne. La moyenne glissante permet de lisser la taille des lignes en diminuant l'importance des extrèmes

Tempo à supprimer avant de rendre : [info pour plus de blabla/ reformuler](https://www.educatim.fr/tq/co/Module_TQ_web/co/moyenne_glissante.html)

In [ ]:
dsetGL = pd.read_fwf("https://www.philippe-fournier-viger.com/spmf/datasets/kosarak_sequences.txt", sep=" ",header=None,)
df_gl= dsetGL[0].str.split(' ', expand=True)

## Cas où le dataset est chargé dans un df
On va calculer la moyenne du nombre d'item par ligne et en fonction de cela on suprimera les colonnes supérieurs à la taille moyenne du nombre d'item/ligne
On ne regarde pas toute les lignes on en regarde que 40% (on pourras changer)

In [ ]:
df_gl.shape 

(990002, 103)

In [ ]:
def resizeDataSet(df,percentage):
  shape = df.shape
  sum = 0
  #for index, row in df.iterrows():
  for _ in range(int(shape[0]*percentage)):
    #if random.uniform(0.,100.) < 5.:
    sum += shape[1] - df.iloc[random.randint(0,shape[0]-1)].isnull().sum()
  mean = sum/(shape[0]*percentage)
  for i in range(shape[1]-1,int(mean),-1):
    del df[i]
  return df

In [ ]:
resizeDataSet(dataset_3,0.4)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26
0,6,-1,9,-1,4,-1,10,-1,3,-1,10,-1,5,-1,10,-1,4,-1,-2,None,None,None,None,None,None,None,None
1,13,-1,7,-1,13,-1,7,-1,13,-1,7,-1,13,-1,7,-1,13,-1,7,-1,13,-1,14,-1,13,-1,7
2,1,-1,4,-1,7,-1,1,-1,10,-1,1,-1,2,-1,1,-1,6,-1,1,-1,-2,None,None,None,None,None,None
3,9,-1,7,-1,9,-1,7,-1,9,-1,12,-1,6,-1,10,-1,13,-1,14,-1,12,-1,3,-1,-2,None,None
4,4,-1,10,-1,2,-1,10,-1,12,-1,10,-1,7,-1,1,-1,10,-1,-2,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31785,9,-1,13,-1,8,-1,3,-1,14,-1,9,-1,13,-1,3,-1,14,-1,9,-1,-2,None,None,None,None,None,None
31786,6,-1,7,-1,6,-1,7,-1,6,-1,7,-1,6,-1,7,-1,6,-1,-2,None,None,None,None,None,None,None,None
31787,1,-1,12,-1,1,-1,10,-1,1,-1,2,-1,11,-1,4,-1,11,-1,6,-1,11,-1,4,-1,1,-1,7
31788,1,-1,2,-1,12,-1,2,-1,7,-1,1,-1,12,-1,7,-1,4,-1,12,-1,-2,None,None,None,None,None,None


## Cas où l'on n'a pas encore chargé le df (je sais pas si c'est pertinant)
On va réaliser un moyenne glissante et pour toute ligne de taille supérieur à la moyenne glissante on réduit la ligne au nombre d'item moyen.